In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/var/folders/03/z9jnwdmd3v744jw4kxc5p0vr0000gn/T/ipykernel_22536/3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [8]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("../data/kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [11]:
# Your code
feature = data['text']
target = data['label']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split( feature, target, test_size=0.3, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)








Train set: (700,) (700,)
Test set: (300,) (300,)


## Data Preprocessing

In [12]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [14]:
# Your code
import re

# Function to clean HTML
def clean_html(html_content):
    # 1. Remove inline JavaScript/CSS
    html_no_js_css = re.sub(r'<(script|style)[^>]*>.*?</\1>', '', html_content, flags=re.DOTALL)
    # 2. Remove HTML comments
    html_no_comments = re.sub(r'<!--.*?-->', '', html_no_js_css, flags=re.DOTALL)
    # 3. Remove remaining HTML tags
    clean_text = re.sub(r'<[^>]+>', '', html_no_comments)
    return clean_text.strip()

# Assuming X_train is a pandas Series or a list
X_train_cleaned = X_train.apply(clean_html)

print(X_train_cleaned[:5])

715    This is a very rough first cut at the CGI spee...
920    Dear Sir,I am Chief Dr. Phil E. Bernard, a nat...
295    Arlen Specter would like to speak to either of...
83     FROM=3AMRS=2E M SESE-SEKODEAR FRIEND=2CI AM MR...
942    Got it.On Mon May 4 2009 at 4:08 PM H  wrote:O...
Name: text, dtype: object


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [15]:
# Your code
# Function to clean text
def clean_text(text):
    # 1. Remove all special characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # 2. Remove all numbers
    text = re.sub(r'\d+', '', text)
    # 3. Remove all single characters
    text = re.sub(r'\b[a-zA-Z]\b', '', text)
    # 4. Remove single characters from the start
    text = re.sub(r'^\s*[a-zA-Z]\s+', '', text)
    # 5. Substitute multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    # 6. Remove prefixed 'b' (if the input is byte-like or mistakenly prefixed)
    text = re.sub(r'^b\s+', '', text)
    # 7. Convert to lowercase
    text = text.lower()
    return text.strip()

# Assuming X_train is a pandas Series or list of text data
X_train_cleaned = X_train_cleaned.apply(clean_text)

print(X_train_cleaned[:5])

715    this is very rough first cut at the cgi speech...
920    dear siri am chief dr phil bernard native of c...
295    arlen specter would like to speak to either of...
83     fromamrse sesesekodear friendci am mrse mariam...
942    got iton mon may at pm wroteok me india raised...
Name: text, dtype: object


## Now let's work on removing stopwords
Remove the stopwords.

In [16]:
# Your code
stop_words = set(stopwords.words('english'))

# Function to remove stopwords
def remove_stopwords(text):
    words = text.split()  # Split text into words
    words = [word for word in words if word not in stop_words]  # Filter out stopwords
    return ' '.join(words)  # Join words back into a single string

# Apply the function to X_train_cleaned
X_train_no_stopwords = X_train_cleaned.apply(remove_stopwords)

print(X_train_no_stopwords[:5])


715    rough first cut cgi speech mostly give idea st...
920    dear siri chief dr phil bernard native cape to...
295    arlen specter would like speak either regardin...
83     fromamrse sesesekodear friendci mrse mariam se...
942    got iton mon may pm wroteok india raised issue...
Name: text, dtype: object


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [19]:
# Your code
from nltk.stem import WordNetLemmatizer
import nltk

# Download WordNet if not already downloaded
nltk.download('wordnet')
nltk.download('omw-1.4')  # For WordNet lemmatizer dependencies

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to apply lemmatization
def lemmatize_text(text):
    words = text.split()  # Break sentences into words
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]  # Lemmatize each word
    return ' '.join(lemmatized_words)  # Reconstruct the sentence

# Apply the function to X_train_no_stopwords
X_train_lemmatized = X_train_no_stopwords.apply(lemmatize_text)

# Output the lemmatized training data
print(X_train_lemmatized[:5])

[nltk_data] Downloading package wordnet to /Users/ambrose/nltk_data...
[nltk_data] Downloading package omw-1.4 to /Users/ambrose/nltk_data...


715    rough first cut cgi speech mostly give idea st...
920    dear siri chief dr phil bernard native cape to...
295    arlen specter would like speak either regardin...
83     fromamrse sesesekodear friendci mrse mariam se...
942    got iton mon may pm wroteok india raised issue...
Name: text, dtype: object


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [20]:
# Your code
from collections import Counter

# Combine all text into a single string
all_text = ' '.join(X_train_lemmatized)

# Split the text into individual words
words = all_text.split()

# Count the frequency of each word
word_counts = Counter(words)

# Extract the top 10 most common words
top_10_words = word_counts.most_common(10)

# Output the results
print("Top 10 Words:", top_10_words)

Top 10 Words: [('money', 609), ('account', 552), ('bank', 533), ('fund', 495), ('u', 455), ('business', 353), ('country', 317), ('transaction', 283), ('company', 280), ('million', 271)]


## Extra features

In [ ]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€",r"\$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 

data_train.head()

## How would work the Bag of Words with Count Vectorizer concept?

In [27]:
# Your code
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the CountVectorizer
vectorizer = CountVectorizer(max_features=800)

# Fit the model and transform the data
X = vectorizer.fit_transform(X_train_lemmatized).toarray()

as_df = pd.DataFrame(X,columns=vectorizer.get_feature_names_out())
as_df.head()




,abandoned,abidjan,able,abroad,ac,accept,acceptance,accord,according,account,...,would,write,writing,yahoo,year,youc,youe,youi,yr,zimbabwe
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,1,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## TF-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [25]:
# Your code
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

count_vectorizer = CountVectorizer (min_df = 1)#(min_df=0.1,max_df=0.7)
tfidf = TfidfTransformer(norm="l2")

term_freq_matrix = count_vectorizer.fit_transform(X_train_lemmatized)
tfidf.fit(term_freq_matrix)
tf_idf_matrix = tfidf.transform(term_freq_matrix)

print ("Vocabulary:", count_vectorizer.vocabulary_,'\n')
print(X_train_lemmatized)
print()
print (tf_idf_matrix.todense())

import pandas as pd
# print idf values
df_idf = pd.DataFrame(tfidf.idf_, index=count_vectorizer.get_feature_names_out(),columns=["idf_weights"])
 
# sort ascending
df_idf.sort_values(by=['idf_weights'])

Vocabulary: {'rough': 11698, 'first': 5241, 'cut': 3481, 'cgi': 2476, 'speech': 12611, 'mostly': 8710, 'give': 5811, 'idea': 6559, 'structure': 12796, 'theme': 13317, 'thesubstance': 13422, 'long': 7954, 'way': 14671, 'go': 5834, 'still': 12734, 'waiting': 14586, 'feedback': 5135, 'input': 6915, 'various': 14454, 'people': 10199, 'including': 6740, 'jackwe': 7283, 'much': 8818, 'improved': 6699, 'draft': 4263, 'monday': 8604, 'nightim': 9228, 'sure': 12933, 'washington': 14641, 'ny': 9433, 'fax': 5103, 'whitehaven': 14829, 'case': 2383, 'thanks': 13154, 'dear': 3598, 'siri': 12355, 'chief': 2556, 'dr': 4261, 'phil': 10292, 'bernard': 1813, 'native': 9026, 'cape': 2332, 'town': 13839, 'southafrica': 12560, 'executive': 4867, 'accountant': 134, 'thesouth': 13417, 'african': 415, 'department': 3773, 'mining': 8504, 'natural': 9028, 'resourcesplease': 11497, 'accept': 95, 'apology': 1002, 'using': 14389, 'medium': 8333, 'toconvey': 13661, 'transactionbusiness': 13890, 'magnitude': 8061, 'b

,idf_weights
money,2.259203
fund,2.305484
million,2.354011
business,2.376358
account,2.416709
...,...
havewatched,6.859361
havewilled,6.859361
haveyou,6.859361
hate,6.859361


## And the Train a Classifier?

In [ ]:
# Your code

### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to **find the most relevant features**.

For example, you can test the following options and check which of them performs better:
- Using "Bag of Words" only
- Using "TF-IDF" only
- Bag of Words + extra flags (money_mark, suspicious_words, text_len)
- TF-IDF + extra flags


You can work with teams of two persons (recommended).

In [ ]:
# Your code